# Intermediate Python
### Patrick Loeber, python-engineer.com
### https://www.youtube.com/watch?v=HGOBQPFzWKo
(3:14:20)
September 17, 2022

## DECORATOR FUNCTIONS:
a function that takes another function as an argument and extends the behavior of the function without explicitly modifying it. It allows you to add new functionality to an already existing function.

Two kinds: function and class

Functions are first class objects, meaning they can be defined inside another function, passed as an argument to another function, or returned from another function.


In [16]:
# The way decorators look:

@my_decorator
def do_something():
    pass

NameError: name 'my_decorator' is not defined

In [17]:
# Writing a decorator function:

def start_end_decorator(function):
    # Wraps the function and defines the extension of it
    def wrapper():
        print('Start')
        function()   # call it as it was passed, not function name
        print('End')
    return wrapper

def print_name():
    print('Alex')

In [18]:
print_name()

Alex


In [19]:
print_name = start_end_decorator(print_name)

print_name()

Start
Alex
End


In [23]:
# Now we can use the decorator with the @ symbol with
# another function

@start_end_decorator
def print_evan():
    print("Evan Marie is AWESOME!")

In [25]:
print_evan()

Start
Evan Marie is AWESOME!
End


### Decorators with funcitons that take arguments:
Because the wrapper function from start_end_decorator takes no arguments, it cannot be used on a function that does take arguments. But it can be rewritten as follows.

We also want to be able to return results from functions we use the decorator on. So we have to create a variable to hold the result of a function and then return that result within the wrapper section of the decorator, otherwise any function we use the decorator on will return None.

In [38]:
def start_end_decorator(function):
    # Wraps the function and defines the extension of it
    def wrapper(*args, **kwargs):
        print('Start')
        result = function(*args, **kwargs)
        print('End')
        return result
    return wrapper

In [39]:
@start_end_decorator
def add5(x):
    return x+5

In [41]:
 print(add5(10))

Start
End
15


###  IDENTITY of the FUNCTION:
This has still not been taken care of. As you see below, Python is confused between the inner wrapper function and the add5() function we used the decorator on. So we will fix that by importing functools and using a decorator that preserves the identity of our function.

In [43]:
print(help(add5))
print(add5.__name__)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    # Wraps the function and defines the extension of it

None
wrapper


In [47]:
import functools

In [53]:
def start_end_decorator(function):
    # This decorator within our decorator preserves
    # the function name of the function fed to it
    @functools.wraps(function)
    def wrapper(*args, **kwargs):
        print('Start')
        result = function(*args, **kwargs)
        print('End')
        return result
    return wrapper

In [50]:
@start_end_decorator
def add10(x):
    return x+5

print(help(add10))
print(add5.__name__)

Help on function add10 in module __main__:

add10(x)

None
wrapper


## This code below is now a template that can be used for any function decorator I want to write

In [52]:
# DECORATOR TEMPLATE:

def my_decorator(function):
    @functools.wraps(function)
    def wrapper(*args, **kwargs):
        # Put something here to do before the function
        result = function(*args, **kwargs)
        # Put something here to do after the function
        return result
    return wrapper

### Decorator that takes an argument:

In [63]:
# This one requires an extra level of function/decorator around
# the inner parts in order to have the argument num_times

def repeat(num_times):
    def decorator_repeat(function):
        @functools.wraps(function)
        def wrapper2(*args, **kwargs):
            for _ in range(num_times):
                result = function(*args, **kwargs)
            return result
        return wrapper2
    return decorator_repeat

@repeat(3)
def greet(name):
    print(f'Hello, {name}!')

In [64]:
@repeat(4)
def add12(x):
    print(x+12)

In [65]:
add12(3)

15
15
15
15


In [73]:
# NESTING DECORATORS

# Original star_end_decorator
def start_end_decorator(function):
    @functools.wraps(function)
    def wrapper(*args, **kwargs):
        print('Start')
        result = function(*args, **kwargs)
        print('End')
        return result
    return wrapper

# debug decorator that extracts the name and arguments of a function
# prints the information of the function, executes, and prints the
# information about the return value

def debug(function):
    @functools.wraps(function)
    def wrapper3(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f'{k} = {v!r}' for k, v in kwargs.items()]
        signature = ','.join(args_repr + kwargs_repr)
        print(f"Calling {function.__name__}({signature})")
        result3 = function(*args, **kwargs)
        print(f'{function.__name__!r} returned {result3!r}')
        return result3
    return wrapper3


# Applying more than 1 decorator executes decorators in the
# order they are given

@debug
@start_end_decorator
def say_hello(name):
    greeting = f'Hello {name}!'
    print(greeting)
    return greeting



In [74]:
say_hello('Evan')

Calling say_hello('Evan')
Start
Hello Evan!
End
'say_hello' returned 'Hello Evan!'


'Hello Evan!'

## -> CLASS DECORATORS:
Similar to function decorators, but typically used for the purpose of updating and maintaining a state.

In [77]:
# This decorator will be used to keep track of how many times
# the function has been called, thus the updating and maintaining.

class CountCalls:
    def __init__(self, function):
        self.function = function
        # Creating a state
        self.num_calls = 0

    # __call__ is necessary for class decorators, which
    # makes objects of the class callable
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f'This is executed {self.num_calls} times.')
        return self.function(*args, **kwargs)


@CountCalls
def say_hello():
    print('Hello')

say_hello()

This is executed 1 times.
Hello


### Typical use cases:
* debugging to print out info about function and arguments
* calculating execution time of a function
* check decorator to see if the arguments fulfill requirements and behave accordingly
* register functions like plugins
* cache return values
* add information or update state.